<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.875 · Deep Learning · Práctica</p>
<p style="margin: 0; text-align:right;">2020-2 · Máster Universitario en Ciencia de Datos (MUDS)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Iñaki Galech Amillano-MSDS 2020/21 <a href="mailto:igalech@uoc.edu">igalech@uoc.edu</a></p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

Este notebook no forma parte de la entrega final de la práctica 1, si bien se ha utilizado para determinar qué modelos pudieran ser los más adecuados para resolver el <u>problema o desafío **original** de Kaggle</u>.

<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
<strong>Nombre y apellidos:</strong> Iñaki Galech Amillano - MSDS 2020/21  <a href="mailto:igalech@uoc.edu">igalech@uoc.edu</a>
</div>

In [ ]:
# %pylab inline
import cv2
import glob
import os
import matplotlib.pylab as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import keras
import keras_preprocessing
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import tensorflow_addons as tfa
import albumentations
import shutil

Cargamos los datos 'originales' del problema en Kaggle.

In [ ]:
p_train = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/train.csv')

En primer lugar mostramos las etiquetas iniciales en número mediante un gráfico.

In [ ]:
# Gráfico inicial del conteo de etiquetas
plt.figure(figsize=(10,8))
labels = sns.barplot(x=p_train.labels.value_counts().index, y=p_train.labels.value_counts());
# Labels, incluidos multilables
for item in labels.get_xticklabels():
    item.set_rotation(90);

# Para imprimir los valores por columnas
for i, v in enumerate(p_train.labels.value_counts()):
    #print (i, v)
    labels.text(i-0.25, v , str(v), color='darkblue', fontsize=10, fontweight='bold')
plt.title('Número de imágenes por clase', fontsize =14, color = 'darkblue');

Debido a la sobreimposición de etiquetas no es posible distinguir multietiquetas en algunos casos. Para resolver este problema separamos las multietiquetas aplicando un `string.split()` a cada instancia.

In [ ]:
p_train['labels'] = p_train['labels'].apply(lambda string: string.split(' '))

A partir de aquí, podemos encodificar las etiquetas de cada imagen con un [MultiLabelBinarizer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html) de Sklearn

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Convertimos el array a una lista
s = list(p_train['labels'])
mlb = MultiLabelBinarizer()
# Construimos un dataframe con el resultado de aplicar el MLB con las nuevas clases como columnas con el mismo índice que p_train.
trainx = pd.DataFrame(mlb.fit_transform(s), columns=mlb.classes_, index=p_train.index)
listadoetiquetas = list(mlb.classes_)

Juntamos imágenes y nuevas etiquetas y mostramos unos cuantos registros.

In [ ]:
labels = pd.concat([p_train['image'], trainx], axis=1)
# Mostramos cómo quedan tras la binarización.
labels.head(3)

Graficamos de nuevo por clases

In [ ]:
# Nueva gráfica una vez tenemos agrupadas las labels en MultiLabels para cada imagen.
labelslista = list(trainx.sum().keys())
label_counts = trainx.sum().values.tolist()

fig, ax = plt.subplots(1,1, figsize=(10,6))

sns.barplot(x= labelslista, y= label_counts, ax=ax)

for i,j in enumerate(label_counts):
    ax.text(i-0.15, j, str(j), color='darkblue', fontsize=10, fontweight='bold');

plt.title('Núevo Total de imágenes de cada clase', fontsize =14, color = 'darkblue');

A continuación se muestran las cifras anteriores en cuanto a sus frecuencias relativas.

In [ ]:
plt.figure(figsize=(6,6))
plt.title("Frecuencias relativas en % para las clases (labels)", fontsize = 14, color = 'darkblue')
#labels = list(trainx.sum().keys())
#label_counts = trainx.sum().values.tolist()
#sorted_counts = round(p_train['labels'].value_counts()/len(p_train['labels'])*100,2)

sorted_counts = list(np.round(np.array(label_counts)/sum(label_counts),2))
plt.pie(sorted_counts, labels = labelslista, startangle = 30,
        counterclock = False, autopct='%.2f%%', textprops={'fontsize':'14'}, normalize=False)
#plt.axis('square')
plt.show();

Pasamos a mostrar cinco imágenes de cada clase. En este caso mostramos cada clase incluidas las 'multiclase'. Es decir, si una imágen tiene más de una clase asociada la mostramos cinco veces, independientemente de si la clase individual se muestra por separado con otras cinco imágenes. La idea es mostrar cómo de cercanas o diferentes (visualmente) son las imágenes asignadas con multietiquetas respecto a aquéllas imágenes con una sola etiqueta.

In [ ]:
from PIL import Image
pathimagenes = '../input/plant-pathology-2021-fgvc8/train_images/'
indices = []
imagenes = []
etiquetas = []

while len(etiquetas) < len(labelslista):
    for i, j in zip(p_train['image'],p_train['labels']):
        if len([x for x in etiquetas if x==j]) < 5:
            #if ((j in etiquetas)==False):
            img = cv2.imread(pathimagenes + i, 1)
            imagenes.append(img)
            etiquetas.append(j)

fig = plt.figure(figsize=(24, 24))
fig.suptitle("5 imágenes de cada clase (label)", fontsize = 14);
for i in range(len(imagenes)):
    ax = plt.subplot(10, 10, i+1)
    plt.imshow(imagenes[i])
    plt.title(etiquetas[i], fontsize = 8)
    plt.axis("off")
plt.show();

Juntamos en un dataframe las imágenes con las nuevas etiquetas. Como vemos ahora tenemos un problema multiclase-multilabel. Este problema es algo diferente al de nuestra práctica, pero sin embargo los modelos nos proporcionarán información valiosa sobre qué modelos y en qué circunstancias se acoplan mejor a resolver nuestro problema.

In [ ]:
labels2 = labels.drop(columns = 'image', axis =1)
df_entreno = pd.concat([p_train, labels2], axis=1)
#df_entreno.head(3)

In [ ]:
# Así es cómo quedan ahora.
df_entreno.head(3)

Para la estrategia de test y validación se subdivide el conjunto de entrenamiento mediante un generador de imágenes a partir del directorio Keras donde está alojadas las imágenes. Como veremos hemos empleado dos directorios. Por un lado se comenzaron los tests con el directorio de Kaggle.com `../kaggle/input/plant-pathology-2021-fgvc8/train_images/` donde se encuentran las imagenes originales. Aumentar las imágenes originales lleva asociado un coste computacional significativo dado el número y tamaño de las imágenes. Para agilizar los tests Kaggle dispone de colaboradores que ya han reducido el tamaño de las imagenes originales a diversos tamaños (256x256, 512x512, etc.) y nos podemos beneficiar de un proceso ya establecido, simplemente haciendo referencia a su autor, Ankur Singh en [resized-plant2021](https://www.kaggle.com/ankursingh12/resized-plant2021) el cual podemos simplemente añadir en Kaggle con `+Add data` en la esquina superior derecha de la plataforma.

De esta forma construimos nuestos juegos de entrenamiento y validación a partir de estos conjuntos ya disminuidos (rescaled) en tamaño, lo cual facilita el entrenamiento muy considerablemente. Si tomamos las imágenes originales los modelos que probamos más adelante (ResNet50, EfficientNet, etc.) tardan del orden de 40 minutos por epoch, mientras que con el tamaño reducido cada epoch tarda 5 o 6 minutos (con GPUs). Dado que el número de horas gratuitas es limitado se opta por las pruebas con tamaños 256x256 con un split o división al 80-20 entre entrenamiento y validación. Este parámetro lo establecemos al construir los generadores de datasets de Keras `ImageDataGenerator()` donde establecemos, además de otros parámetros, la división train-validación (80/20) como vemos `validation_split = 0.2`

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255.,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split = 0.2,
    zoom_range = 0.2,
    shear_range = 0.2,
    vertical_flip = False)

test_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255.,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split = 0.2,
    zoom_range = 0.2,
    shear_range = 0.2,
    vertical_flip = False)

<u>Estrategia de aumento de datos</u>  
En este punto hay que considerar de forma combinada los resultados obtenidos en cuanto a la binarización de las multilabels y también el tamaño y calidad de las imágenes. Por el enunciado sabemos que tenemos aproximadadmente 23000 imágenes en alta resolución en formato RGB, las cuales además según se indica incluyen: diferentes iluminaciones, diferentes contextos y fondo de la imagen, obtenidas en diferentes estaciones y horas del día y a diferentes niveles de maduración de las manzanas. Además las cámaras que han tomado las fotografías son de muy diversa índole (no es la misma cámara ni son del mismo tipo). En resumen, imágenes muy grandes con una gran resolución pero nada uniformes en cuanto a sus parámetros.
El objetivo por tanto es hacer uso en lo posible de una estandarización de tamaño que sea computacionalmente razonable reduciendo en la medida de lo posible el tamaño de la imagen, pero al mismo tiempo manteniendo la no uniformidad en la posición y contexto de las fotografías para forzar a nuestros modelos a aprender condiciones muy variadas que son las que realmente se dan en el mundo agrícola.
Es también importante resaltar que el objetivo es categorizar imágenes con enfermedades, es decir, teniendo en cuenta que una imagen (una hoja) puede presentar varias enfermedades el objetivo es categorizarla con *al menos* una enfermedad. El motivo es que si el agricultor sabe que un determinado árbol tiene una enfermedad irá a inspeccionarlo y de esta forma podrá determinar 'in-situ' si además presenta otro tipo de patología además de la indicada por nuestro modelo.

Del enunciado de la competición: "The main objective of the competition is to develop machine learning-based models to accurately classify a given leaf image from the test dataset to a particular disease category, and to identify an individual disease from multiple disease symptoms on a single leaf image." 

Con lo comentado se opta por elegir imágenes reducidas a 256x256 en tres canales RGB y sobre las cuales se emplearán técnicas de aumentado de datos (data augmentation). Para ello se usa de nuevo la librería de imágenes de Keras (images) dónde a nuestro generador de imágenes obligamos a realizar: rotaciones, ensanchado y estirado, volteos horizontales, zoom  y desenfoque (shear) con los parámetros que se muetran a continuación. Esto producirá imagenes 

<u>Tipo de modelo y tamaño de imagen de entrada</u>  
Por lo expuesto anteriormente se entrenan tres modelos ya paraemtrizados y entrenados: ResNet50, EfficientNetB0, y . Los modelos elegidos están especialmente contrastados en problemas multietiquetas como nuestro caso. 

In [ ]:
labelslista = list([str(i) for i in labelslista])
#columns=["desert", "mountains", "sea", "sunset", "trees"]

In [ ]:
df_entreno2 = df_entreno.drop(['labels'], axis=1)

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

train_data = train_datagen.flow_from_dataframe(
    df_entreno2,
    directory= '/kaggle/input/resized-plant2021/img_sz_256/', # originales en 'plant-pathology-2021-fgvc8/train_images/'
    x_col = "image",
    y_col=  labelslista,
    color_mode="rgb",
    #target_size = (256,256),
    #class_mode="categorical",
    batch_size=58,
    subset = "training",
    shuffle=True,
    seed=27,
    class_mode='raw'
)

validacion_data = train_datagen.flow_from_dataframe(
    df_entreno2,
    directory= '/kaggle/input/resized-plant2021/img_sz_256/', # originales en 'plant-pathology-2021-fgvc8/train_images/'
    x_col = "image",
    y_col=  labelslista,
    color_mode="rgb",
    #target_size = (256,256),
    #class_mode="categorical",
    batch_size=54,
    subset = "validation",
    shuffle=True,
    seed=27,
    class_mode='raw'
)

# Este último set no implementa shuffle (shuffle=False) y se usará para las 
# predicciones y así poder comparar las clases de las predicciones vs. clases 
# verdaderas.
set_validacion_ordenado = train_datagen.flow_from_dataframe(
    df_entreno,
    directory = '/kaggle/input/resized-plant2021/img_sz_256/',
    x_col="image",
    y_col = labelslista,
    #target_size=(256, 256),
    color_mode="rgb",
    #class_mode="categorical",
    batch_size=54,
    shuffle=False,
    seed=27,
    subset='validation',
    interpolation="nearest",
    class_mode='raw'
)

set_test = test_datagen.flow_from_directory(
    directory = '/kaggle/input/plant-pathology-2021-fgvc8/',
    batch_size=1,
    # No tenemos y_col porque son las que tenemos que predecir
    target_size=(256, 256),
    classes = ['test_images'],
    color_mode="rgb",
    class_mode=None,
    shuffle=False,
    seed=27,
#    interpolation="nearest",
)

## <u>Construcción y entrenamiento de 6 modelos</u>:
    1.-Resnet50v2
    2.-InceptionV3
    3.-EfficientNetB0
    4.-Resnet50
    5.-InceptionResNetV2
    6.-DenseNet121

### <u>Modelo ResNet50V2:</u>

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
#from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
#from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input
from tensorflow.keras.models import Sequential

def create_model():
    
    pretrained = ResNet50V2(include_top=False, weights='imagenet',input_shape=[256,256, 3])
    # pretrained = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=[256, 256, 3])
            
    x = pretrained.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    #x = tf.keras.layers.Dense(64, activation='relu')(x)
    #x = tf.keras.layers.Dense(16, activation='relu')(x)
    # Obsérvese aquí que la activación es sigmoid
    outputs = tf.keras.layers.Dense(len(labelslista), activation="sigmoid", dtype='float32')(x)
        
    model = tf.keras.Model(pretrained.input, outputs, name='modeloResNetV2')
    return model

model = create_model()

In [ ]:
# Check del nombre del modelo
#model.name

In [ ]:
# Función que compilará el modelo con SGD y la learning rate como parámetro. 
# Nótese que la función de pérdida es BinaryCrossentropy porque **no** queremos excluir clases unas de otras (Multilabel). 
# También introducimos nuestra medidad como F1-score 'macro' 

def compile_model(model, lr=0.03):
    
    #optimizer = tf.keras.optimizers.Adam(lr=lr)
    optimizer = tf.keras.optimizers.SGD(lr=0.03, decay=1e-4, momentum=0.8, nesterov=True)
    
    loss = tf.keras.losses.BinaryCrossentropy()
    
    # tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy')
    metrics = [
    tfa.metrics.F1Score(num_classes = len(labelslista), average = "macro",name = "f1_score")
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

Los siguientes parámetros son comunes a todos los modelos. Introducen un 'checkpoint' para guardar los mejores modelos según se van obteniendo máximos en nuestra función (f1-score en validación). También definimos un EarlyStop con 5 epochs de paciencia y un parámetro que actúa como reductor del learning rate si se producen situaciones de estancamiento ([`ReduceLROnPlateau()`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau))

In [ ]:
METRIC = "val_f1_score"

def create_callbacks(metric = METRIC):
    
    guardar_path = './mejor_modelo_origKaggle_'+ model.name + '.h5'
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath = guardar_path,
        monitor = metric,
        mode ='max',
        save_best_only=True,
        verbose=1,
    )

    reducccionlr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor = metric,
        mode = 'max',
        factor = 0.1,
        patience = 3,
        verbose = 0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor = metric,
        mode ='max',
        patience = 5, 
        verbose = 1
    )
    
    callbacks = [checkpoint, reducccionlr, earlystop]         
    
    return callbacks

In [ ]:
EPOCHS = 10
VERBOSE = 1

tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    
    model = create_model()
    model = compile_model(model, lr=0.03)
   
    callbacks = create_callbacks()
    
    history = model.fit(
                        train_data,
                        epochs=EPOCHS,
                        callbacks=callbacks,
                        validation_data = validacion_data,
                        verbose=VERBOSE
                       )

Guardamos los resultados de cada entrenamiento en un dataframe y a apartir de éste lo volcamos en un fichero CSV que iremos subiendo al dataset de Kaggle.com que entregamos con la práctica.

In [ ]:
resultados = pd.DataFrame(history.history)

In [ ]:
modeloResNetV2 = 'modeloResNetV2_history05282021Kaggle.csv'
with open(modeloResNetV2, mode='w') as f:
    resultados.to_csv(f)

## <u>Modelo InceptionV3:</u>

In [ ]:
try: # TPUs, si tenemos habilitado el cluster y generamos archivos tipo .tfrec
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)

except ValueError: # GPUs
    strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
    #strategy = tf.distribute.get_strategy() # una CPU y una GPU
    #strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy() # for clusters of multi-GPU machines

print("Number of accelerators: ", strategy.num_replicas_in_sync)

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input

def create_InceptionV3():
    
    pretrained = InceptionV3(include_top=False, weights='imagenet',input_shape=[256, 256, 3], pooling = 'avg')
    x = pretrained.output
    #x = tf.keras.layers.GlobalAveragePooling2D()(x) # No necesitamos pooling porque lo ponemos en la salida del EfficientNetB0
    outputs = tf.keras.layers.Dense(len(validacion_data.class_indices),activation="sigmoid", dtype='float32')(x)
    model = tf.keras.Model(pretrained.input, outputs, name = 'modeloInceptionV3')
    return model

model = create_InceptionV3()


def compile_model(model, lr=0.0001):
    
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    
    loss = tf.keras.losses.CategoricalCrossentropy()
    
    #tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy')
    metrics = [
    tfa.metrics.F1Score(num_classes = len(validacion_data.class_indices), average = "macro",name = "f1_score")
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

In [ ]:
model.name

In [ ]:
EPOCHS= 10
VERBOSE =1

tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    
    #model = create_model()
    model = compile_model(model, lr=0.0001)
   
    callbacks = create_callbacks()
    
    historyIncepcionV3 = model.fit(
                        train_data,
                        epochs=EPOCHS,
                        callbacks=callbacks,
                        validation_data = validacion_data,
                        verbose=VERBOSE
                       )

In [ ]:
resultadosInceptionV3 = pd.DataFrame(historyIncepcionV3.history)
modeloInceptionV3_cvs_file = 'resultadosInceptionV3_history.csv'
with open(modeloInceptionV3_cvs_file, mode='w') as f:
    resultadosInceptionV3.to_csv(f)

## <u>Modelo EfficientNetB0:</u>

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

def create_EffNetB0model():
    
    pretrained = EfficientNetB0(include_top=False, weights='imagenet',input_shape=[256, 256, 3], pooling = 'avg')
    x = pretrained.output
    #x = tf.keras.layers.GlobalAveragePooling2D()(x) # No necesitamos pooling porque lo ponemos en la salida del EfficientNetB0
    outputs = tf.keras.layers.Dense(6,activation="sigmoid", dtype='float32')(x)
    model = tf.keras.Model(pretrained.input, outputs, name = 'modeloEffNetB0')
    return model

#model = create_EffNetB0model()


def compile_model(model, lr=0.0001):
    
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    
    loss = tf.keras.losses.BinaryCrossentropy()
    
    #tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy')
    metrics = [
    tfa.metrics.F1Score(num_classes = 6, average = "macro",name = "f1_score")
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

In [ ]:
EPOCHS = 10
VERBOSE = 1

tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    
    model = create_EffNetB0model()
    model = compile_model(model, lr=0.0001)
 
    callbacks = create_callbacks()
    
    history = model.fit(
                    train_data,
                    epochs=EPOCHS,
                    callbacks=callbacks,
                    validation_data = validacion_data,
                    verbose = VERBOSE
                   )

In [ ]:
resultadosEffNetB0 = pd.DataFrame(history.history)

In [ ]:
resultadosEffNetB0_csv_file = 'resultadosEffNetB0_history2.csv'
with open(resultadosEffNetB0_csv_file, mode='w') as f:
    resultadosEffNetB0.to_csv(f)

## <u>Modelo RestNet50:</u> 
Creamos y entrenamos el modelo RestNet50 con el mismo set de datos, mismos parámetros y guardamos tanto el modelo como los resultados del entrenamiento.

In [ ]:
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input

def create_RestNet50():
    
    pretrained = ResNet50(include_top=False, weights='imagenet',input_shape=[256, 256, 3])
    x = pretrained.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x) 
    # Para RestNet50 debemos usar softmax si utilizamos los pesos pre-entrenados.
    outputs = tf.keras.layers.Dense(6,activation="sigmoid", dtype='float32')(x)
    model = tf.keras.Model(pretrained.input, outputs, name = 'modeloRestNet50')
    return model

model = create_RestNet50()


def compile_model(model):
    
    #optimizer = tf.keras.optimizers.Adam(lr=lr)
    optimizer = tf.keras.optimizers.SGD(lr=0.03, decay=1e-4, momentum=0.8, nesterov=True)

    
    loss = tf.keras.losses.BinaryCrossentropy()
    
    #tf.keras.metrics.CategoricalAccuracy(name='categorical_accuracy')
    metrics = [
    tfa.metrics.F1Score(num_classes = 6, average = "macro",name = "f1_score")
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

Entrenamos el modelo RestNet50

In [ ]:
EPOCHS = 10
VERBOSE = 1

tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    
    callbacks = create_callbacks()
    model = compile_model(model)
    historyResNet50 = model.fit(
                    train_data,
                    epochs=EPOCHS,
                    callbacks=callbacks,
                    validation_data = validacion_data,
                    verbose = VERBOSE
                   )

In [ ]:
resultadosRestNet50 = pd.DataFrame(historyResNet50.history)
modeloResNet50 = 'modeloResNet50_history2.csv'
with open(modeloResNet50, mode='w') as f:
    resultadosRestNet50.to_csv(f)

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
#tf.keras.applications.densenet.DenseNet121

seed = 1200
tf.random.set_seed(seed)

#base_model = tf.keras.applications.DenseNet121(weights=weight_path, include_top=False, pooling='avg')
base_model = tf.keras.applications.DenseNet121(input_shape=(256,256,3),
                                               include_top=False,
                                               weights='imagenet',
                                               pooling='avg')
#pesos_entrenados = '../input/keras-pretrained-models/'
#model = keras.applications.InceptionResNetV2(weights=pesos_entrenados, include_top=False, input_shape=(256, 256, 3))
base_model.trainable = False

x = base_model.output
#fully connected layer
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(16, activation='relu')(x)
# finally, the softmax for the classifier 
predictions = tf.keras.layers.Dense(6, activation='sigmoid')(x)
modeloDenseNet121 = tf.keras.Model(base_model.inputs, predictions, name='DenseNet121')


In [ ]:
resultados = pd.read_csv('/kaggle/input/resultadosmodelosprac1gpu/modelos_Resultados.csv')

In [ ]:
 resultados.modelo

In [ ]:
resultados[resultados['modelo']=='EfficeintNetB0']['epoch'].to_list()

In [ ]:
seed = 1200
tf.random.set_seed(seed)

pesos_entrenados = '../input/keras-pretrained-models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
model = keras.applications.InceptionResNetV2(weights=pesos_entrenados, include_top=False, input_shape=(256, 256, 3))
# Tamaños entrada-salida del modelo InceptionResnet50v2
#print(model.input)
#print(model.output)

In [ ]:
modelo_sobre_IncRSNet50V2 = tf.keras.Sequential([
    model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(len(labelslista), 
    kernel_initializer=keras.initializers.RandomUniform(seed=seed),
    bias_initializer=keras.initializers.Zeros(), name='dense_top', activation='sigmoid')
])

# Freezing the weights
for layer in modelo_sobre_IncRSNet50V2.layers[:-1]:
    layer.trainable=False
    
modelo_sobre_IncRSNet50V2.summary()

In [ ]:
VERBOSE = 10
f1 = tfa.metrics.F1Score(num_classes=6, average='macro')

#callbacks = keras.callbacks.EarlyStopping(monitor=f1, patience=3, mode='max', restore_best_weights=True)


modelo_sobre_IncRSNet50V2.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=keras.optimizers.Adam(lr=1e-4), 
              metrics= [f1])

with tf.device('/device:GPU:0'):
    
    callbacks = create_callbacks()
    #model = compile_model(model, lr=0.0001)
    history_IncRSNet50V2 = modelo_sobre_IncRSNet50V2.fit(
                    train_data,
                    epochs=10,
                    callbacks=callbacks,
                    validation_data = val_data,
                    verbose = VERBOSE
                   )

#modelo_sobre_IncRSNet50V2.fit(train_data, epochs=20, callbacks=callbacks)

In [ ]:
resultadosIncepRSNet50V2 = pd.DataFrame(history_IncRSNet50V2.history)
modeloIncepResNet50v2 = 'modeloIncepRSNet50V2_history.csv'
with open(modeloIncepResNet50v2, mode='w') as f:
    resultadosIncepRSNet50V2.to_csv(f)

## Modelo DenseNet121

In [ ]:
# Probamos un DenseNet121
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
#tf.keras.applications.densenet.DenseNet121

seed = 43
tf.random.set_seed(seed)

#base_model = tf.keras.applications.DenseNet121(weights=weight_path, include_top=False, pooling='avg')
base_model = tf.keras.applications.DenseNet121(input_shape=(256,256,3),
                                               include_top=False,
                                               weights='imagenet',
                                               pooling='avg')
x = base_model.output
#fully connected layer
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(16, activation='relu')(x)
# finally, the softmax for the classifier 
output = tf.keras.layers.Dense(6, activation='sigmoid')(x)
model = tf.keras.Model(base_model.inputs, output, name='DenseNet121')


In [ ]:
optimizer = keras.optimizers.SGD(lr=0.03, decay=1e-4, momentum=0.8, nesterov=True)


In [ ]:
EPOCHS = 20
VERBOSE = 1
f1 = tfa.metrics.F1Score(num_classes=6, average='macro')

tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):

    callbacks = create_callbacks()
    model = compile_model(model)
    history_DenseNet121 = model.fit(
                    train_data,
                    epochs=EPOCHS,
                    callbacks=callbacks,
                    validation_data = validacion_data,
                    verbose = VERBOSE
                   )

In [ ]:
resultadosDenseNet121= pd.DataFrame(history_DenseNet121.history)
modeloDenseNet121 = 'modeloDenseNet121_history2.csv'
with open(modeloDenseNet121, mode='w') as f:
    resultadosDenseNet121.to_csv(f)

## Predicciones (pruebas con el problema original Kaggle).

In [ ]:
test_generator.reset()
predicciones = model.predict_generator(test_generator, verbose=1)

In [ ]:
test_generator = test_datagen.flow_from_directory(
)

## <u>Gráficas preliminares de resultados a partir de las pruebas con el problema original Kaggle </u>
Se muestra a continuación la evolución de la pérdida y de F1-Score para los conjuntos de entrenamiento y de validación para nuestros cuatro modelos.

In [ ]:
fig, ((ax1,ax2, ax3, ax4), (ax5, ax6, ax7, ax8)) = plt.subplots(2,4, figsize=(24,8))
plt.suptitle("Gráficas de pérdida y f1-score para los modelos \n (Sets entrenamientor y validación)", color='darkblue')
plt.style.use('seaborn')

ax1.plot(resultados[resultados['modelo']=='ResNet50']['loss'], color='blue')
ax1.plot(resultados[resultados['modelo']=='ResNet50']['val_loss'], color='orange')
ax1.set_xticks(resultados[resultados['modelo']=='ResNet50']['epoch'])
ax1.legend(['pérdida entren.', 'perdida val.'])
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Pérdida (loss)')
ax1.set_title("Pérdida Modelo ResNet50")
               
ax2.plot(resultados[resultados['modelo']=='ResNet50']['f1_score'].to_list(),color='maroon')
ax2.plot(resultados[resultados['modelo']=='ResNet50']['val_f1_score'].to_list(),color='lime')
ax2.legend(['f1-score entren.', 'f1-score valid.'])
ax2.set_xticks(resultados[resultados['modelo']=='ResNet50V2']['epoch'].to_list())
ax2.set_xticklabels(resultados[resultados['modelo']=='ResNet50V2']['epoch'].to_list())
ax2.legend(['f1-score entren.', 'f1-score valid.'])
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Pérdida (loss)')
ax2.set_title("f1-score (macro) Modelo ResNet50")


ax3.plot(resultados[resultados['modelo']=='ResNet50V2']['loss'].to_list(),color='blue')
ax3.plot(resultados[resultados['modelo']=='ResNet50V2']['val_loss'].to_list(),color='orange')
ax3.set_xticks(resultados[resultados['modelo']=='ResNet50V2']['epoch'].to_list())
ax3.set_xticklabels(resultados[resultados['modelo']=='ResNet50V2']['epoch'].to_list())
ax3.legend(['pérdida entren.', 'perdida val.'])
ax3.set_xlabel('Epochs')
ax3.set_ylabel('Pérdida (loss)')
ax3.set_title("Pérdida Modelo ResNet50v2")

ax4.plot(resultados[resultados['modelo']=='ResNet50V2']['f1_score'].to_list(),color='maroon')
ax4.plot(resultados[resultados['modelo']=='ResNet50V2']['val_f1_score'].to_list(),color='lime')
ax4.legend(['f1-score entren.', 'f1-score valid.'])
ax4.set_xticks(resultados[resultados['modelo']=='ResNet50V2']['epoch'].to_list())
ax4.set_xticklabels(resultados[resultados['modelo']=='ResNet50V2']['epoch'].to_list())
ax4.set_xlabel('Epochs')
ax4.set_ylabel('Pérdida (loss)')
ax4.set_title("f1-score (macro) Modelo ResNet50v2")


ax5.plot(resultados[resultados['modelo']=='EfficeintNetB0'][['loss', 'val_loss']])
ax5.set_xticklabels(resultados[resultados['modelo']=='EfficeintNetB0']['epoch'].to_list())
ax5.legend(['pérdida entren.', 'perdida val.'])
#ax5.set_xticks(resultados[resultados['modelo']=='EfficeintNetB0']['epoch'].to)
ax5.set_xlabel('Epochs')
ax5.set_ylabel('Pérdida (loss)')
ax5.set_title("Pérdida Modelo EfficeintNetB0")

ax6.plot(resultados[resultados['modelo']=='EfficeintNetB0'][['f1_score', 'val_f1_score']])
ax6.set_xticklabels(resultados[resultados['modelo']=='EfficeintNetB0']['epoch'].to_list())
ax6.legend(['f1-score entren.', 'f1-score valid.'])
ax6.set_xlabel('Epochs')
ax6.set_ylabel('f1-score')
ax6.set_title("f1-score (macro) Modeloo EfficeintNetB0")

ax7.plot(resultados[resultados['modelo']=='InceptionV3'][['loss', 'val_loss']])
ax7.set_xticklabels(resultados[resultados['modelo']=='InceptionV3']['epoch'].to_list())
ax7.legend(['pérdida entren.', 'perdida val.'])
ax7.set_xlabel('Epochs')
ax7.set_ylabel('Pérdida (loss)')
ax7.set_title("Pérdida Modelo InceptionV3")

ax8.plot(resultados[resultados['modelo']=='InceptionV3'][['f1_score', 'val_f1_score']])
ax8.set_xticklabels(resultados[resultados['modelo']=='InceptionV3']['epoch'].to_list())
ax8.legend(['f1-score entren.', 'f1-score valid.'])
ax8.set_xlabel('Epochs')
ax8.set_ylabel('f1-score')
ax8.set_title("f1-score (macro) Modeloo InceptionV3")
fig.tight_layout(h_pad=5, w_pad=5)
plt.show();

A continuación obtenemos las predicciones con el mejor modelo

In [ ]:
val_generator_ej4_sin_shuffle = data_generator_ej4.flow_from_directory(
  directory='/k /drive/MyDrive/UCMerced_LandUse/Images/',
  target_size=(224, 224),
  #color_mode="rgb",
  class_mode='categorical',
  seed = 42,
  shuffle = False,
  follow_links=False,
  subset='validation')